<a href="https://colab.research.google.com/github/sjnam95/Test002/blob/master/Project_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 필요한 모듈을 설치합니다
!pip install mxnet-cu101
!pip install gluonnlp pandas tqdm
!pip install sentencepiece==0.1.85
!pip install transformers
!pip install torch #원래 ==1.3.1

#SKT에서 공개한 KoBERT 모델을 불러옵니다 
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

     |████████████████████████████████| 356.7MB 48kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 348kB 14.6MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595698 sha256=f2592be1b562250c2cc1904262a391af29cc643d91c2e8999cb596f5906ab146
  Stored in directory: /root/.cache/pip/wheels/37/65/52/63032864a0f31a08b9a88569f803b5bafac8abd207fd7f7534
Successfully built gluonnlp
     |████████████████████████████████| 1.0MB 13.0MB/s 
     |████████████████████████████████| 2.5MB 14.4MB/s 
     |████████████████████████████████| 3.3MB 36.9MB/s 
     |████████████████████████████████| 901kB 36.5MB/s 
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-a7ax4qly
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-a7ax4qly
  Created wheel for kob

In [ ]:
import pandas as pd

import torch
from torch import nn

In [ ]:
# import
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
from tqdm.notebook import tqdm

from kobert.utils import get_tokenizer 
from kobert.pytorch_kobert import get_pytorch_kobert_model 

from transformers import AdamW 
from transformers import get_linear_schedule_with_warmup

In [ ]:
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
#GPU 사용 시 cuda
device = torch.device("cuda:0")

In [ ]:
# get model and vocab file
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [ ]:
#koBERT의 토크나이저를 사용합니다
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        output = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        pooler = output[1]
        
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = torch.load('/content/drive/MyDrive/kobert_balance_re.pt')

In [ ]:
# model = BERTClassifier(bertmodel,  dr_rate=0.005).to(device)
# model.load_state_dict(torch.load('/content/drive/MyDrive/kobert_balance.pt'))

In [ ]:
# # 위에서 설정한 tok, max_len, batch_size, device를 그대로 입력
# # comment : 예측하고자 하는 텍스트 데이터 리스트
# def getSentimentValue(comment, tok, max_len, batch_size, device):
#   commnetslist = [] # 텍스트 데이터를 담을 리스트
#   emo_list = [] # 감성 값을 담을 리스트
#   for c in comment: # 모든 사용자 입력문장(사용자 감정 입력값)
#     commnetslist.append( [c, 5] ) # [사용자 입력 문장, 임의의 양의 정수값] 설정
    
#   pdData = pd.DataFrame( commnetslist, columns = [['Sentence', 'Emotion']] )
#   pdData = pdData.values
#   test_set = BERTDataset(pdData, 0, 1, tok, max_len, True, False) 
#   test_input = torch.utils.data.DataLoader(test_set, batch_size=batch_size, num_workers=5)
  
#   for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_input):
#     token_ids = token_ids.long().to(device)
#     segment_ids = segment_ids.long().to(device)
#     valid_length= valid_length 
#     # 이때, out이 예측 결과 리스트
#     out = model(token_ids, valid_length, segment_ids)
	
#     # e는 2가지 실수 값으로 구성된 리스트
#     # 0번 인덱스가 더 크면 부정, 긍정은 반대
#     for e in out:
#       if e[0]>e[1]: # 부정
#         value = 0
#       else: #긍정
#         value = 1
#       emo_list.append(value)

#   return emo_list # 텍스트 데이터에 1대1 매칭되는 감성값 리스트 반환

In [ ]:
df_test = pd.read_excel('/content/test1.xlsx')

In [ ]:
# 해당 데이터에 대해 분류를 시작합니다
model.eval()
answer=[]
train_acc = 0.0
test_acc = 0.0
for batch_id, (token_ids, valid_length, segment_ids) in enumerate(tqdm_notebook('test1.xlsx')):
  token_ids = token_ids.long().to(device)
  segment_ids = segment_ids.long().to(device)
  valid_length= valid_length
  out = model(token_ids, valid_length, segment_ids)
  max_vals, max_indices = torch.max(out, 1)
  answer.append(max_indices.cpu().clone().numpy())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


ValueError: ignored

In [ ]:
emotion = ['Angry', 'Neutral', 'Happiness', 'Sadness', 'Fear', 'Surprise', 'Disgust']
emotion_dict = {num:key for num,key in enumerate(emotion)}

a_emotion = []
for a_emotion_array in answer:
  a_emotion += [emotion_dict[i] for i in a_emotion_array]

In [ ]:
answer_df['A_Emotion'] = a_emotion
answer_df.to_excel('./result.xlsx')

In [ ]:
# def data_perprocess(data) : 
#   raw = re.split

In [ ]:
# def data_preprocess(data):
#     raw = re.split('[\r\n\.\?\!]', data)
#     #raw = data.replace('\r\n', ' ').replace('.', ' ').replace('\?', ' ')
#     text = []
    
#     for val in raw:
#         if val == '':
#             continue
#         text.append([val, 0.0])
    
    
#     print(text)
#     return text

In [ ]:
# def predict(model, text):
#     device = torch.device("cuda:0")
#     max_len = 64
#     batch_size = 64
#     warmup_ratio = 0.1
#     num_epochs = 2
#     max_grad_norm = 1
#     log_interval = 200 
#     learning_rate =  5e-5
    
#     tokenizer = get_tokenizer()
#     bertmodel, vocab = get_pytorch_kobert_model()
#     tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)
#     data_test = BERTDataset(text, 0, 1, tok, max_len, True, False)
#     test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=0)
#     model.eval()
    
#     answer=[]
#     for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
#         token_ids = token_ids.long().to(device)
#         segment_ids = segment_ids.long().to(device)
#         valid_length= valid_length
#         label = label.long().to(device)
#         out = model(token_ids, valid_length, segment_ids)
#         max_vals, max_indices = torch.max(out, 1)
#         answer.append(max_indices.cpu().clone().numpy())
    
#     result = F.softmax(out)
    
#     print(result)
#     return result
    


In [ ]:
def calc_result(result):
    Angry = 0.0
    Neutral = 0.0
    Happiness = 0.0
    Sadness = 0.0
    Fear = 0.0
    Surprise = 0.0
    Disgust = 0.0
    result = result.detach().cpu().clone().numpy()
    
    for data in result:
        Angry += data[0] 
        Neutral += data[1] 
        Happiness += data[2]
        Sadness += data[3]
        Fear += data[4]
        Surprise += data[5]
        Disgust += data[6]
    
    result = [Angry, Neutral, Happiness, Sadness, Fear, Surprise, Disgust]
    
    
    results = normalize(result)
    results = [0.1 if x == 0.0 else x for x in results]
    results = {'Angry': results[0], 'Neutral': results[1], 'Happiness': results[2], 'Sadness': results[3], 'Sadness': results[4], 'Surprise': results[5], 'Disgust': results[6]}
    
    return results